In [7]:
import cv2
import dlib
import numpy as np
import os
import csv
from deepface import DeepFace

def classify_eye_size(eye_area, threshold=650):
    # 根據面積分類眼睛大小
    if eye_area > threshold:
        return 'big'
    else:
        return 'small'

def analyze_emotion(img_path):
    # 使用 DeepFace 分析表情
    try:
        result = DeepFace.analyze(img_path=img_path, actions=['emotion'], enforce_detection=False)
        if result: 
            emotions = result[0]['emotion']
            highest_emotion = max(emotions, key=emotions.get)
            return highest_emotion
        else:
            return "No Face Detected"
    except Exception as e:
        return str(e)

def mouth_open(mouth):
    # 判斷嘴巴是否開合
    vertical_dist = np.linalg.norm(mouth[7] - mouth[2])
    horizontal_dist = np.linalg.norm(mouth[0] - mouth[4])
    ratio = float(vertical_dist) / float(horizontal_dist)
    return ratio > 0.8

# 初始化 dlib 的臉部檢測器和特徵點檢測器
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')  # 更換為您的特徵點模型路徑

# 初始化 Haar Cascade 分類器
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# 指定資料夾路徑
folder_path = 'human_picture'  # 更改為您的圖片資料夾路徑

# CSV 檔案路徑
csv_file = 'Human.csv'

# 遍歷資料夾中的所有圖片
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Image', 'Eye Size', 'Highest Emotion', 'Mouth State'])  # 寫入標題

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            faces = detector(gray)
            eye_sizes = []
            mouth_state = "No Face Detected"

            for face in faces:
                landmarks = predictor(gray, face)
                # 眼睛大小分類
                roi_gray = gray[face.top():face.bottom(), face.left():face.right()]
                eyes = eye_cascade.detectMultiScale(roi_gray)
                for (ex,ey,ew,eh) in eyes:
                    eye_area = ew * eh
                    eye_size = classify_eye_size(eye_area)
                    eye_sizes.append(eye_size)

                # 嘴巴開合檢測
                mouth_points = np.array([[p.x, p.y] for p in landmarks.parts()[48:68]])
                mouth_state = "open" if mouth_open(mouth_points) else "close"

            emotion_result = analyze_emotion(img_path)

            # 將結果寫入 CSV 檔案
            if eye_sizes:
                writer.writerow([filename, eye_sizes[0], emotion_result, mouth_state])
            else:
                writer.writerow([filename, "No Eyes Detected", emotion_result, mouth_state])

print("處理完成，結果已儲存到", csv_file)


Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 30.12it/s]

處理完成，結果已儲存到 Human.csv
